In [54]:
from pyspark.sql import SQLContext
sqlContext =SQLContext(sc)
df = sqlContext.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |-- text: string (nullable = true)



In [55]:
df.registerTempTable("tweets")
result_data=sqlContext.sql("SELECT * from tweets limit 10")
result_data.show()

+--------------------+--------------------+--------------------+--------------------+
|                 _id|          created_at|            entities|                text|
+--------------------+--------------------+--------------------+--------------------+
|[591eb2306e5f1f40...|Fri May 19 08:52:...|    [WrappedArray()]|Justice prédictiv...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|[WrappedArray([FD...|RT @GrosBill92: @...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|    [WrappedArray()]|@GG_RMC @MLP_offi...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|[WrappedArray([Bo...|RT @RMCinfo: @Emm...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|[WrappedArray([Lé...|RT @JSmi20: #Légi...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|[WrappedArray([Pr...|RT @MaximCombes: ...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|    [WrappedArray()]|Super vidéo du ca...|
|[591ebc356e5f1f75...|Fri May 19 09:33:...|    [WrappedArray()]|Avec Macron, le r...|
|[591ebed26e5f1f75...|Fri May 19 09:45:...|    [Wrappe

In [56]:
df.select("text").show()

+--------------------+
|                text|
+--------------------+
|Justice prédictiv...|
|RT @GrosBill92: @...|
|@GG_RMC @MLP_offi...|
|RT @RMCinfo: @Emm...|
|RT @JSmi20: #Légi...|
|RT @MaximCombes: ...|
|Super vidéo du ca...|
|Avec Macron, le r...|
|@EmmanuelMacron i...|
|@Valeurs Quelle e...|
+--------------------+



In [57]:
rdd1=df.rdd

In [58]:
rdd1.collect()

[Row(_id=Row(oid=u'591eb2306e5f1f4051255ae1'), created_at=u'Fri May 19 08:52:00 +0000 2017', entities=Row(hashtags=[]), text=u'Justice pr\xe9dictive : le "big data" frappe \xe0 la porte des tribunaux et des cabinets d\'avocats https://t.co/ussrlylqEA via @franceinfo'),
 Row(_id=Row(oid=u'591ebc356e5f1f755ad461bd'), created_at=u'Fri May 19 09:33:47 +0000 2017', entities=Row(hashtags=[Row(text=u'FDE04'), Row(text=u'EnMarcheBenelux'), Row(text=u'LREM'), Row(text=u'legislatives2017')]), text=u'RT @GrosBill92: @PAAngade rencontre concitoyens \xe0 Bruxelles. #FDE04 #EnMarcheBenelux #LREM #legislatives2017 https://t.co/mHACY1Js4H'),
 Row(_id=Row(oid=u'591ebc356e5f1f755ad461be'), created_at=u'Fri May 19 09:33:48 +0000 2017', entities=Row(hashtags=[]), text=u'@GG_RMC @MLP_officiel @etienneliebig Les deux Etienne....les deux...!'),
 Row(_id=Row(oid=u'591ebc356e5f1f755ad461bf'), created_at=u'Fri May 19 09:33:48 +0000 2017', entities=Row(hashtags=[Row(text=u'BourdinBDV')]), text=u"RT @RMCinfo: @Em

In [59]:
rdd2=df.map(lambda r:[r.text,r.entities.hashtags])

In [60]:
rdd2.collect()

[[u'Justice pr\xe9dictive : le "big data" frappe \xe0 la porte des tribunaux et des cabinets d\'avocats https://t.co/ussrlylqEA via @franceinfo',
  []],
 [u'RT @GrosBill92: @PAAngade rencontre concitoyens \xe0 Bruxelles. #FDE04 #EnMarcheBenelux #LREM #legislatives2017 https://t.co/mHACY1Js4H',
  [Row(text=u'FDE04'),
   Row(text=u'EnMarcheBenelux'),
   Row(text=u'LREM'),
   Row(text=u'legislatives2017')]],
 [u'@GG_RMC @MLP_officiel @etienneliebig Les deux Etienne....les deux...!',
  []],
 [u"RT @RMCinfo: @EmmanuelMacron 2nd d\xe9bat #BourdinBDV \xe0 9h30 \U0001f4e2 : la direction de @France2tv a-t-elle raison d'\xe9vincer David Pujadas ? Votez !\u2026",
  [Row(text=u'BourdinBDV')]],
 [u'RT @JSmi20: #L\xe9gislatives NE VOUS LAISSEZ PAS IMPRESSIONNER PAR"L\'ATTRAPE-VOIX" #HULOT DE MACRON:IL NE POURRA PAS METTRE SA POL\u2026 ',
  [Row(text=u'L\xe9gislatives'), Row(text=u'HULOT')]],
 [u'RT @MaximCombes: Le Ni de droite ni de gauche vu par @rodho_g \n#PremierMinistre #Macron #gouuvernementph

In [100]:
from textblob_fr import PatternTagger, PatternAnalyzer
from textblob import TextBlob
from textblob import Blobber

In [63]:
text1=u"Valeurs Quelle est votre position Monsieur"

In [64]:
blob1 = TextBlob(text1, pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [65]:
blob1.tags

[(u'Valeurs', u'NNS'),
 (u'Quelle', u'NNP'),
 (u'est', u'VB'),
 (u'votre', u'PRP$'),
 (u'position', u'NN'),
 (u'Monsieur', u'NNP')]

In [66]:
blob1.sentiment

(0.0, 0.0)

In [68]:
tb =Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

In [69]:
blob2 = tb(u"Quelle belle matinée")
blob2.sentiment

(0.8, 0.8)

In [70]:
blob3 = tb(u"c'est une voiture terrible")
blob3.sentiment

(-0.7, 0.6)

In [86]:
blob5 = tb(u"Avec Macron, le renouvellement social attendra!\naux l\xe9gislatives, il a surtout investi des cadres sup' et des entre\u2026 https://t.co/7m2Mcbu92k")
blob5.sentiment

(0.3125, 0.1)

In [88]:
rdd3=rdd2.map(lambda x:tb(x[0]))
rdd3.take(3)

[TextBlob("Justice prédictive : le "big data" frappe à la porte des tribunaux et des cabinets d'avocats https://t.co/ussrlylqEA via @franceinfo"),
 TextBlob("RT @GrosBill92: @PAAngade rencontre concitoyens à Bruxelles. #FDE04 #EnMarcheBenelux #LREM #legislatives2017 https://t.co/mHACY1Js4H"),
 TextBlob("@GG_RMC @MLP_officiel @etienneliebig Les deux Etienne....les deux...!")]

In [94]:
rdd_texttb=rdd3.collect()

In [104]:
for i in rdd_texttb:
    i.sentiment
    print(i,i.sentiment)

(TextBlob("Justice prédictive : le "big data" frappe à la porte des tribunaux et des cabinets d'avocats https://t.co/ussrlylqEA via @franceinfo"), (0.0, 0.0))
(TextBlob("RT @GrosBill92: @PAAngade rencontre concitoyens à Bruxelles. #FDE04 #EnMarcheBenelux #LREM #legislatives2017 https://t.co/mHACY1Js4H"), (0.0, 0.0))
(TextBlob("@GG_RMC @MLP_officiel @etienneliebig Les deux Etienne....les deux...!"), (0.0, 0.0))
(TextBlob("RT @RMCinfo: @EmmanuelMacron 2nd débat #BourdinBDV à 9h30 📢 : la direction de @France2tv a-t-elle raison d'évincer David Pujadas ? Votez !…"), (0.0, 0.0))
(TextBlob("RT @JSmi20: #Législatives NE VOUS LAISSEZ PAS IMPRESSIONNER PAR"L'ATTRAPE-VOIX" #HULOT DE MACRON:IL NE POURRA PAS METTRE SA POL… "), (0.0, 0.0))
(TextBlob("RT @MaximCombes: Le Ni de droite ni de gauche vu par @rodho_g 
#PremierMinistre #Macron #gouuvernementphilippe https://t.co/QfuszmbPzg"), (0.0050000000000000044, 0.1))
(TextBlob("Super vidéo du candidat @FranceInsoumise des Français d'Europe du Nord sur

In [ ]:
rdd4=rdd3.map(lambda x:x.sentiment)